## Packages

In [1]:
import pandas as pd
import re

## Load data set

In [2]:
file_name = "PtLanka-final.csv"

word_df = pd.read_csv(file_name)
#word_df = word_df[["SriLanka_creole","phonetic_transcription"]]
word_df["Portuguese_IPA"] = word_df["Portuguese_IPA"].fillna("")
word_df

,Unnamed: 0,SriLanka_creole,phonetic_transcription,Portuguese_definition,Portuguese_IPA,English_definition,pos,Related_languages,pt_related annotations,sources,original_full_entry,original_Portuguese_definition,original_linguistic_notes,page
0,0,Abaço,abaso,abaixo,ɐ.ˈbaj.ʃu,below,adverb,NaN,NaN,NaN,"Abaço, abaixo. P. us. V.",abaixo,pouco usado,135.0
1,1,Aberçà,abeɾsaː,abraçar,ɐ.bɾɐ.ˈsaɾ,hug,verb,NaN,NaN,"O Bruffador, ii periódico","Aberçà, abersà, aburçá {Br. 2), abraçar.",abraçar,NaN,135.0
2,2,abersà,abeɾsaː,abraçar,ɐ.bɾɐ.ˈsaɾ,hug,verb,NaN,NaN,"O Bruffador, ii periódico","Aberçà, abersà, aburçá {Br. 2), abraçar.",abraçar,NaN,135.0
3,3,aburçá,abuɾsaː,abraçar,ɐ.bɾɐ.ˈsaɾ,hug,verb,NaN,NaN,"O Bruffador, ii periódico","Aberçà, abersà, aburçá {Br. 2), abraçar.",abraçar,NaN,135.0
4,4,Aburcé,abuɾseː,aborrecer,ɐ.bu.Rɨ.ˈseɾ,annoy,verb,NaN,NaN,NaN,Aburcé. Mais us. burcé.,Mesmo que burcé,NaN,135.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2517,2517,Wine,ʋine,vinho europeu,,european wine,noun,English,NaN,NaN,NaN,vinho europeu,NaN,NaN
2518,2518,Zumbá,zumbaː,zombar,,mock,verb,NaN,NaN,NaN,"Zumba, zombar. Zombe-rías, zombrias, zombàrios...",zombar,NaN,182.0
2519,2519,Zomberías,zombeɾiːas,zombarias,,mockery,noun,NaN,NaN,NaN,NaN,zombarias,NaN,NaN
2520,2520,zombrias,zombɾias,zombarias,,mockery,noun,NaN,NaN,NaN,NaN,zombarias,NaN,NaN


# Convert to TRSVG

Before syllable parsing, we replaced the phonemes by larger categories:

- T: corresponds to obstruents (consonants like T, which typically occur as first unit of an onset only);

- R: corresponds to sonorants (consonants like R, which typically occur as onset- first or second unit- or coda);

- V: corresponds to vowels;

- G: corresponds to glides.

In [3]:
word_list = list(word_df["Portuguese_IPA"].apply(str))
word_list

['ɐ.ˈbaj.ʃu',
 'ɐ.bɾɐ.ˈsaɾ',
 'ɐ.bɾɐ.ˈsaɾ',
 'ɐ.bɾɐ.ˈsaɾ',
 'ɐ.bu.Rɨ.ˈseɾ',
 '',
 'ɐ.bu.Rɨ.ˈsi.du',
 'aw.ˈzẽ.sjɐ',
 '',
 '',
 '',
 'ˈka',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'ɐ.ˈsow.ɡɨ',
 '',
 '',
 '',
 'ɐ.ˈdewʃ',
 'ˈdow.tu',
 '',
 '',
 '',
 '',
 '',
 '',
 'ɐ.duɫ.tˈɛ.ɾi.u',
 'ɐ.duɫ.tˈɛ.ɾi.u',
 '',
 '',
 '',
 '',
 '',
 'ɐ.fɐj.ˈsɐ̃w̃',
 '',
 'a.gwɐ',
 '',
 '',
 '',
 'ɐ.ɡɾɐ.ˈda.vɛɫ',
 'ˈvwaɾ',
 'ˈvwaɾ',
 'ɐ.ˈĩ.dɐ',
 '',
 '',
 '',
 '',
 'ˈaɾ.vu.ɾɨ',
 'ˈaɾ.vu.ɾɨ',
 'ˈaɾ.vu.ɾɨ',
 '',
 'aɫ.ˈdɐj.ɐ',
 'ɐ.ˈlɛ.ɡɾɨ',
 'ɐ.ˈlɛ.ɡɾɨ',
 '',
 '',
 'aɫ.mu.ˈfa.dɐ',
 'aɫ.ˈfɐ̃.dɨ.ɡɐ',
 'aɫ.fɐj.ˈa.tɨ',
 '',
 '',
 '',
 '',
 'aɫ.ˈɡɐ̃j̃',
 '',
 '',
 '',
 '',
 '',
 'ɐ.ˈlɛ.ɡɾɨ',
 'ɐ.ˈlɛ.ɡɾɨ',
 'ɐ.ˈlɛ.ɡɾɨ',
 'ɐ.lɨ.ˈɡɾi.ɐ',
 'ɐ.lɨ.ˈɡɾi.ɐ',
 'ɐɾ.ˈma.ɾju',
 'ɐɾ.mɐ.ˈʒɐ̃j̃',
 'aɫ.ˈmo.su',
 '',
 '',
 'ɐ.ˈlwa.du',
 'i.lu.mi.ˈnaɾ',
 'ɐ.vɨɾ.ˈsɐ̃w̃',
 'aɫ.vu.ɾu.ˈsaɾ',
 'aɫ.vu.ɾu.ˈsaɾ',
 'aɫ.vu.ˈɾo.su',
 'aɫ.vu.ˈɾo.su',
 '',
 '',
 'a.mɐ.ˈɲɐ̃',
 'a.mɐ.ˈɲɐ̃',
 'a.mɐ.ˈɲɐ̃',
 'a.mɐ.ˈɲɐ̃',
 'ɐ.mu.ˈɾo.zu',
 'ɐ.mu.ˈɾ

In [4]:
#word_list = [re.sub("\.", "", i) for i in word_list]
word_list = [re.sub("ˈ", "", i) for i in word_list]

In [5]:
word_list = [re.sub("s", "C", i) for i in word_list]
word_list = [re.sub("g", "C", i) for i in word_list]
word_list = [re.sub("k", "C", i) for i in word_list]
word_list = [re.sub("v", "C", i) for i in word_list]
word_list = [re.sub("ɲ", "C", i) for i in word_list]
word_list = [re.sub("ʎ", "C", i) for i in word_list]
word_list = [re.sub("z", "C", i) for i in word_list]
word_list = [re.sub("p", "C", i) for i in word_list]
word_list = [re.sub("b", "C", i) for i in word_list]
word_list = [re.sub("t", "C", i) for i in word_list]
word_list = [re.sub("m", "C", i) for i in word_list]
word_list = [re.sub("R", "C", i) for i in word_list]
word_list = [re.sub("n", "C", i) for i in word_list]
word_list = [re.sub("lʲ", "C", i) for i in word_list]
word_list = [re.sub("f", "C", i) for i in word_list]
word_list = [re.sub("ɡ", "C", i) for i in word_list]
word_list = [re.sub("ʁ", "C", i) for i in word_list]
word_list = [re.sub("ɫ", "C", i) for i in word_list]
word_list = [re.sub("r", "C", i) for i in word_list]
word_list = [re.sub("ɾ", "C", i) for i in word_list]
word_list = [re.sub("l", "C", i) for i in word_list]
word_list = [re.sub("ʒ", "C", i) for i in word_list]
word_list = [re.sub("d", "C", i) for i in word_list]
word_list = [re.sub("j", "C", i) for i in word_list]
word_list = [re.sub("t̠ʃ", "C", i) for i in word_list]
#word_list = [re.sub("T̠", "", i) for i in word_list]
word_list = [re.sub("ʃ", "C", i) for i in word_list]

word_list = [re.sub("\̃", "", i) for i in word_list]
word_list = [re.sub("", "", i) for i in word_list]
word_list = [re.sub("ɛ", "V", i) for i in word_list]
word_list = [re.sub("õ", "VC", i) for i in word_list]
word_list = [re.sub("ĩ", "VC", i) for i in word_list]
word_list = [re.sub("ũ", "VC", i) for i in word_list]
word_list = [re.sub("ɔ", "V", i) for i in word_list]
word_list = [re.sub("ô", "V", i) for i in word_list]
word_list = [re.sub("ɨ", "V", i) for i in word_list]
word_list = [re.sub("ɐ", "V", i) for i in word_list]
word_list = [re.sub("ẽ", "V", i) for i in word_list]
word_list = [re.sub("u", "V", i) for i in word_list]
word_list = [re.sub("o", "V", i) for i in word_list]
word_list = [re.sub("a", "V", i) for i in word_list]
word_list = [re.sub("e", "V", i) for i in word_list]
word_list = [re.sub("i", "V", i) for i in word_list]
word_list = [re.sub("ə", "V", i) for i in word_list]
word_list = [re.sub("j", "G", i) for i in word_list]
word_list = [re.sub("w", "G", i) for i in word_list]

word_list = [re.sub("GV", "V.V", i) for i in word_list]

word_list[0:10]

['V.CVC.CV',
 'V.CCV.CVC',
 'V.CCV.CVC',
 'V.CCV.CVC',
 'V.CV.CV.CVC',
 '',
 'V.CV.CV.CV.CV',
 'VG.CV.CCV',
 '',
 '']

In [6]:
word_df = word_df[["SriLanka_creole","Portuguese_definition","Portuguese_IPA"]]
word_df["phono_CV"] = word_list
word_df

C:\Users\CARLOS~1\AppData\Local\Temp/ipykernel_15156/2349386221.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  word_df["phono_CV"] = word_list


,SriLanka_creole,Portuguese_definition,Portuguese_IPA,phono_CV
0,Abaço,abaixo,ɐ.ˈbaj.ʃu,V.CVC.CV
1,Aberçà,abraçar,ɐ.bɾɐ.ˈsaɾ,V.CCV.CVC
2,abersà,abraçar,ɐ.bɾɐ.ˈsaɾ,V.CCV.CVC
3,aburçá,abraçar,ɐ.bɾɐ.ˈsaɾ,V.CCV.CVC
4,Aburcé,aborrecer,ɐ.bu.Rɨ.ˈseɾ,V.CV.CV.CVC
...,...,...,...,...
2517,Wine,vinho europeu,,
2518,Zumbá,zombar,,
2519,Zomberías,zombarias,,
2520,zombrias,zombarias,,


In [7]:
word_df.to_csv("crioulo_IPA_CV_pt.csv", index = False, header = True)